# ε(E) Tunability 계산을 위한 VASP 입력 파일 생성

이 노트북은 Materials Project에서 물질 구조를 가져와서 ε(E) tunability 계산을 위한 VASP 입력 파일들을 자동으로 생성합니다.

## 🎯 목표
- CSV 파일에서 MP ID들을 읽어오기
- 각 물질에 대해 전기장 스윕용 VASP 입력 파일 생성
- PEAD + LCALCEPS 설정으로 유전상수 계산


## 1단계: 필요한 라이브러리 설치 및 임포트


In [84]:
# 필요한 라이브러리들을 설치합니다 (처음 한 번만 실행하면 됩니다)
%pip install pymatgen mp-api

print("라이브러리 설치가 완료되었습니다!")


Note: you may need to restart the kernel to use updated packages.
라이브러리 설치가 완료되었습니다!


In [85]:
# 라이브러리들을 불러옵니다
import os
import csv
import numpy as np  # 수치 계산을 위한 라이브러리
from pathlib import Path  # 파일 경로를 다루기 위한 라이브러리

# pymatgen의 핵심 기능들을 불러옵니다
from pymatgen.core import Structure  # 결정 구조를 다루는 클래스
from pymatgen.io.vasp.sets import MPStaticSet  # VASP 입력 파일 세트 클래스
from pymatgen.io.vasp.inputs import Kpoints  # k-points 설정을 위한 클래스

# Materials Project API를 사용하기 위한 클래스
try:
    from mp_api.client import MPRester as MPClient  # 최신 버전의 API 클라이언트
    print("✅ 최신 mp-api를 사용합니다.")
except ImportError:
    from pymatgen.ext.matproj import MPRester as MPClient  # 구버전 API 클라이언트
    print("✅ 구버전 pymatgen API를 사용합니다.")

print("✅ 라이브러리 로딩이 완료되었습니다!")


✅ 최신 mp-api를 사용합니다.
✅ 라이브러리 로딩이 완료되었습니다!


## 2단계: API 키 설정하기


In [86]:
# 🔑 여기에 당신의 API 키를 직접 입력하세요!
api_key = "0okpcG0Nl7FhbZTDQoxHBFZryxB2SI32"

# API 키가 올바르게 설정되었는지 확인
if api_key and api_key != "your_api_key_here":
    print("✅ API 키가 성공적으로 설정되었습니다!")
    print(f"API 키: {api_key[:10]}...{api_key[-4:]}")  # 보안을 위해 일부만 표시
    print("이제 Materials Project에서 데이터를 가져올 수 있습니다!")
else:
    print("❌ API 키를 올바르게 설정해주세요!")
    print("위의 api_key = '...' 부분에 실제 API 키를 입력하세요.")


✅ API 키가 성공적으로 설정되었습니다!
API 키: 0okpcG0Nl7...SI32
이제 Materials Project에서 데이터를 가져올 수 있습니다!


## 3단계: CSV 파일에서 MP ID 읽어오기


In [87]:
# CSV 파일에서 MP ID들을 읽어오는 함수
def read_mp_ids_from_csv(csv_path):
    """
    CSV 파일에서 Materials Project ID들을 읽어오는 함수
    
    Args:
        csv_path: CSV 파일 경로
        
    Returns:
        MP ID들의 리스트
    """
    mp_ids = []
    
    try:
        with open(csv_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)  # 첫 번째 줄을 헤더로 사용
            
            for row in reader:
                if 'material_id' in row and row['material_id'].strip():
                    mp_ids.append(row['material_id'].strip())
                    
        print(f"✅ CSV 파일에서 {len(mp_ids)}개의 MP ID를 성공적으로 읽어왔습니다!")
        print(f"첫 5개 ID: {mp_ids[:5]}")
        
    except FileNotFoundError:
        print(f"❌ 파일을 찾을 수 없습니다: {csv_path}")
    except Exception as e:
        print(f"❌ CSV 파일 읽기 중 오류 발생: {e}")
    
    return mp_ids

# CSV 파일 경로 설정
csv_file_path = "/Users/jaekwansmac/Desktop/MP_dielectric_constant/perovskite_dielectric_bec_data.csv"

# MP ID들 읽어오기
mp_ids = read_mp_ids_from_csv(csv_file_path)

✅ CSV 파일에서 265개의 MP ID를 성공적으로 읽어왔습니다!
첫 5개 ID: ['mp-1013548', 'mp-1013555', 'mp-1112340', 'mp-1112596', 'mp-1112622']


## 4단계: ε(E) Tunability 계산을 위한 INCAR 설정


In [ ]:
# ===== 구조 불러오기 함수 =====
def get_structure_from_mp(mp_id, api_key):
    """
    Materials Project에서 구조를 가져오는 함수
    
    Args:
        mp_id: Materials Project ID
        api_key: API 키
        
    Returns:
        Structure 객체
    """
    with MPClient(api_key) as mpr:
        structure = mpr.get_structure_by_material_id(mp_id)
    return structure



# ===== 공통 INCAR 설정 =====
# - MPStaticSet을 기반으로 하되, PEAD finite-field + LCALCEPS + 수렴강화
# - Γ-centered 정규 메쉬 필요(PEAD는 Γ 포함 정규 메쉬 요구)
#   (MPStaticSet은 기본적으로 MP 규칙을 따르며, KPOINTS는 자동 생성됨)
common_incar = {
    "LCALCEPS": False,      # c축 단일방향 스윕하려고
    "ISMEAR": 0,           # insulator 가정 (Gaussian smearing)
    "SIGMA": 0.05,         # smearing 폭 (eV)
    "EDIFF": 1e-6,         # zero-field에 대한 수렴 기준
    # 참고: field-polarized 단계에서는 VASP가 EDIFF/100까지 자동으로 더 조입니다.
    # LRPA를 원하면 True (local-field effects 무시, RPA) — 기본은 포함
    # "LRPA": True,
    # 수렴 안정화를 위해 ALGO, PREC, ADDGRID 등을 적절히 조정 가능
    "PREC": "Accurate",    # 정확도 우선
    "ADDGRID": True,       # 그리드 포인트 추가로 수렴 안정화
    "LREAL": False,        # 정확도 우선 (실공간 계산 비활성화)
    "LCHARG": False,       # 디스크 절약 (CHG 파일 저장 안함)
    "LWAVE": False,        # 디스크 절약 (WAVECAR 파일 저장 안함)
    "ENCUT": 520,          # 컷오프 에너지 (eV)
    "NSW": 0,              # 이온 스텝 수 (0 = 이온 이동 안함) 
    "LCALCPOL":True,       # 후처리 (예: P‑E 곡선, 유도량 확인)에 유용      
    "IBRION": -1,
              # 이온 이동 방법 (-1 = 고정)
}

print("✅ INCAR 설정이 완료되었습니다!")
print("주요 설정:")
print(f"  - LCALCEPS: {common_incar['LCALCEPS']} (유전상수 계산)")
print(f"  - ISMEAR: {common_incar['ISMEAR']} (Gaussian smearing)")
print(f"  - EDIFF: {common_incar['EDIFF']} (수렴 기준)")
print(f"  - PREC: {common_incar['PREC']} (정확도)")


✅ INCAR 설정이 완료되었습니다!
주요 설정:
  - LCALCEPS: False (유전상수 계산)
  - ISMEAR: 0 (Gaussian smearing)
  - EDIFF: 1e-08 (수렴 기준)
  - PREC: Accurate (정확도)


## 7단계: 모든 CSV 물질에 대해 일괄 처리하기


In [95]:
# 모든 CSV 물질에 대해 필드 스윕 입력을 생성하는 함수
def create_field_sweep_for_all_materials(mp_ids, max_materials=3, E_mags=[0.001, 0.003, 0.005]):
    """
    CSV에서 읽어온 모든 MP ID에 대해 필드 스윕 입력을 생성하는 함수
    
    Args:
        mp_ids: MP ID들의 리스트
        max_materials: 최대 처리할 물질 수
        E_mags: 전기장 세기 리스트
    """
    if not mp_ids:
        print("❌ 처리할 MP ID가 없습니다!")
        return
    
    # 처리할 물질 수 제한
    materials_to_process = mp_ids[:max_materials]
    print(f"🔧 {len(materials_to_process)}개 물질에 대해 필드 스윕 입력을 생성합니다...")
    
    success_count = 0
    fail_count = 0
    
    for i, mp_id in enumerate(materials_to_process, 1):
        print(f"\\n{'='*60}")
        print(f"📋 {i}/{len(materials_to_process)}: {mp_id} 처리 중")
        print(f"{'='*60}")
        
        try:
            # 구조 가져오기
            structure = get_structure_from_mp(mp_id, api_key)
            print(f"✅ 구조 로드 완료: {structure.composition.reduced_formula}")
            
            # 루트 디렉토리 생성
            root = Path(f"finite_field_sweep_{mp_id}")
            root.mkdir(exist_ok=True)
            
            # k-points 설정 (Γ-centered 정규 메쉬)
            kpoints = Kpoints.gamma_automatic(kpts=(8,8,8))
            print(f"✅ k-points 설정: Γ-centered 8x8x8 메쉬")
            
            # Zero-field 참고용 입력 생성
            zf_dir = root / "E0_ref"
            zf_dir.mkdir(exist_ok=True)
            zf_incar = {**common_incar, "EFIELD_PEAD": [0.0, 0.0, 0.0]}
            vset_zf = MPStaticSet(structure, user_incar_settings=zf_incar, user_kpoints_settings=kpoints)
            vset_zf.write_input(str(zf_dir), potcar_spec=True)
            print(f"✅ Zero-field 참고 입력 생성: {zf_dir.name}")
            
            # ===== c축 방향 스윕만 생성 =====
            # c축 단위벡터 계산 (결정학적 [001] 방향)
            c_hat = structure.lattice.matrix[2] / np.linalg.norm(structure.lattice.matrix[2])
            print(f"✅ c축 단위벡터: [{c_hat[0]:.4f}, {c_hat[1]:.4f}, {c_hat[2]:.4f}]")
            
            total_dirs = 0
            for E in E_mags:
                # c축 방향으로 전기장 벡터 계산: E * c_hat
                evec = (c_hat * E).tolist()
                tag = f"E_{E:.4f}_along_001"
                wdir = root / tag
                wdir.mkdir(parents=True, exist_ok=True)
                
                field_incar = {**common_incar, "EFIELD_PEAD": evec}
                vset = MPStaticSet(structure, user_incar_settings=field_incar, user_kpoints_settings=kpoints)
                vset.write_input(str(wdir), potcar_spec=True)
                
                total_dirs += 1
                print(f"✅ {tag} 생성 완료 (c축 방향)")
            
            print(f"✅ {mp_id} 필드 스윕 입력 생성 완료! ({total_dirs + 1}개 디렉토리)")
            print(f"   - Zero-field: 1개")
            print(f"   - c축 방향: {len(E_mags)}개")
            success_count += 1
                
        except Exception as e:
            print(f"❌ {mp_id} 처리 중 오류: {e}")
            fail_count += 1
    
    print(f"\\n🎉 전체 처리 완료!")
    print(f"✅ 성공: {success_count}개")
    print(f"❌ 실패: {fail_count}개")
    print(f"📊 성공률: {success_count/(success_count+fail_count)*100:.1f}%")

# 사용자 설정
MAX_MATERIALS = 3  # 테스트용으로 3개만 처리 (필요시 더 늘릴 수 있음)
E_FIELD_MAGNITUDES = [0.001, 0.003, 0.005]  # 전기장 세기 (eV/Å)

print(f"⚙️  설정:")
print(f"   - 최대 처리 물질 수: {MAX_MATERIALS}개")
print(f"   - 전기장 세기: {E_FIELD_MAGNITUDES} eV/Å")
print(f"   - 전기장 방향: c축 방향 (결정학적 [001] 방향)")

# 모든 물질에 대해 필드 스윕 입력 생성
if mp_ids:
    create_field_sweep_for_all_materials(
        mp_ids=mp_ids,
        max_materials=MAX_MATERIALS,
        E_mags=E_FIELD_MAGNITUDES
    )
else:
    print("⚠️  먼저 위의 셀을 실행해서 MP ID들을 읽어와주세요!")

⚙️  설정:
   - 최대 처리 물질 수: 3개
   - 전기장 세기: [0.001, 0.003, 0.005] eV/Å
   - 전기장 방향: c축 방향 (결정학적 [001] 방향)
🔧 3개 물질에 대해 필드 스윕 입력을 생성합니다...
\n============================================================
📋 1/3: mp-1013548 처리 중


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 7810.62it/s]


✅ 구조 로드 완료: Ca3SbN
✅ k-points 설정: Γ-centered 8x8x8 메쉬
✅ Zero-field 참고 입력 생성: E0_ref
✅ c축 단위벡터: [0.0000, 0.0000, 1.0000]
✅ E_0.0010_along_001 생성 완료 (c축 방향)
✅ E_0.0030_along_001 생성 완료 (c축 방향)
✅ E_0.0050_along_001 생성 완료 (c축 방향)
✅ mp-1013548 필드 스윕 입력 생성 완료! (4개 디렉토리)
   - Zero-field: 1개
   - c축 방향: 3개
\n============================================================
📋 2/3: mp-1013555 처리 중


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 19599.55it/s]


✅ 구조 로드 완료: Sr3AsN
✅ k-points 설정: Γ-centered 8x8x8 메쉬
✅ Zero-field 참고 입력 생성: E0_ref
✅ c축 단위벡터: [0.0000, -0.0000, 1.0000]
✅ E_0.0010_along_001 생성 완료 (c축 방향)
✅ E_0.0030_along_001 생성 완료 (c축 방향)
✅ E_0.0050_along_001 생성 완료 (c축 방향)
✅ mp-1013555 필드 스윕 입력 생성 완료! (4개 디렉토리)
   - Zero-field: 1개
   - c축 방향: 3개
\n============================================================
📋 3/3: mp-1112340 처리 중


Retrieving MaterialsDoc documents: 100%|██████████| 1/1 [00:00<00:00, 26214.40it/s]

✅ 구조 로드 완료: Cs2TlRhF6
✅ k-points 설정: Γ-centered 8x8x8 메쉬
✅ Zero-field 참고 입력 생성: E0_ref
✅ c축 단위벡터: [0.7071, 0.7071, 0.0000]
✅ E_0.0010_along_001 생성 완료 (c축 방향)
✅ E_0.0030_along_001 생성 완료 (c축 방향)
✅ E_0.0050_along_001 생성 완료 (c축 방향)
✅ mp-1112340 필드 스윕 입력 생성 완료! (4개 디렉토리)
   - Zero-field: 1개
   - c축 방향: 3개
\n🎉 전체 처리 완료!
✅ 성공: 3개
❌ 실패: 0개
📊 성공률: 100.0%


## 🎉 완료! ε(E) Tunability 계산용 필드 스윕 입력 생성

축하합니다! 이제 CSV 파일에서 MP ID들을 읽어와서 ε(E) tunability 계산을 위한 필드 스윕용 VASP 입력 파일들을 자동으로 생성할 수 있습니다.

### 🔧 생성되는 입력 파일들:

1. **Zero-field 참고용**: `E0_ref/` - 전기장이 없는 상태
2. **결정학적 c축 방향 스윕**: 
   - `E_0.0010_along_001/`, `E_0.0030_along_001/`, `E_0.0050_along_001/`
   - 구조가 z축과 정렬되어 있지 않아도 항상 결정학적 [001] 방향으로 전기장 적용

### 📁 각 디렉토리에는:
- **INCAR**: PEAD + LCALCEPS 설정으로 유전상수 계산
- **POSCAR**: Materials Project에서 가져온 원자 구조
- **KPOINTS**: Γ-centered 정규 메쉬 (PEAD 요구사항)
- **POTCAR.spec**: POTCAR 생성에 필요한 정보

### ⚙️ 주요 INCAR 설정:
- `LCALCEPS = True`: field-polarized 상태에서 ε∞ tensor 계산
- `EFIELD_PEAD = [Ex, Ey, Ez]`: 전기장 벡터 (eV/Å)
- `ISMEAR = 0`: 절연체 가정
- `EDIFF = 1e-8`: 엄격한 수렴 기준

### 🚀 다음 단계:
1. 생성된 입력 파일들을 VASP가 설치된 HPC로 복사
2. VASP 환경에서 POTCAR 파일 생성: `potcar.sh $(cat POTCAR.spec)`
3. 각 디렉토리에서 VASP 계산 실행
4. 결과에서 유전상수 텐서 추출
5. 전기장에 따른 유전상수 변화 분석 (ε(E) tunability)
